In [ ]:
import warnings
import logging
import os
import sys

# Suprimir logs de TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Redirigir salida estándar para suprimir warnings en Voila
warnings.filterwarnings('ignore')
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import io

In [2]:
# Cargar el modelo (reemplaza 'modelo_final.h5' por el nombre de tu modelo)
model = load_model('modelo/efficientnet_retina_v1_40_learn_3.0_batch_32_rep_3.h5')

In [31]:
# Widgets
upload_button = widgets.FileUpload(accept='.jpg, .jpeg', multiple=False)
output = widgets.Output()
message_label = widgets.Label(value="Por favor, sube una imagen en formato .jpg o .jpeg")
message_label2 = widgets.HTML(value="")
warning_label = widgets.HTML(value="<h3 style='color:#8e1414;'>Esto es solamente un prototipo que busca un acercamiento a la clasificación de enfermedades oculares que causan ceguera irreversible, especificamente: Glaucoma, Retinopatía diabética y Catarata; mediante imágenes de retinoscopía. Por lo tanto, las respuestas nunca serán 100% precisas.</h3>")
progress = widgets.FloatProgress(value=0, min=0, max=1, description='Progreso:',
                                 layout={'display': 'none'})  # Inicialmente oculta
# Cargar una imagen de la UDI
udi_image_path = 'imagen_udi/Logo-udi-web.png'  # Asegúrate de colocar la ruta correcta
with open(udi_image_path, "rb") as f:
    udi_image = f.read()
    
# Crear el widget para la imagen de la UDI
udi_image_widget = widgets.Image(
    value=udi_image,
    format='jpg',
    width=300,  # Ajusta el tamaño según sea necesario
    height=100
)

In [36]:
# Función para procesar la imagen y realizar la predicción
# Función para procesar la imagen y realizar la predicción
def process_image(change):
    output.clear_output()
    progress.value = 0.0
    progress.layout.display = 'none'  # Asegurar que la barra esté oculta si no se ha subido imagen

    # Verificar si se ha cargado una imagen
    if upload_button.value:
        # Mostrar la barra de progreso cuando empieza el procesamiento
        progress.layout.display = 'flex'
        progress.value = 0.1

        # Obtener el archivo subido
        uploaded_file = upload_button.value[0]
        img = Image.open(io.BytesIO(uploaded_file['content'].tobytes()))

        with output:
            # Mostrar la imagen
            plt.imshow(img)
            plt.axis('off')
            plt.show()

        # Actualizar barra de progreso
        progress.value = 0.5

        # Preprocesar la imagen para el modelo (modificar según tus necesidades)
        img_resized = img.resize((224, 224))  # Ajusta esto al tamaño esperado por tu modelo
        img_array = np.array(img_resized)
        img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión batch

        # Realizar la predicción
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction, axis=1)

        # Actualizar barra de progreso
        progress.value = 0.8

        class_names = {0: "Catarata", 1: "Retinopatía Diabética", 2: "Glaucoma", 3: "Normal"}
        predicted_label = class_names.get(predicted_class[0], "Clase desconocida")

        # Mostrar la predicción del modelo en grande con formato HTML
        message_label2.value = f"<h2 style='color:#046c9c;'>Predicción del modelo: {predicted_label}</h1>"
        
        # Finalizar la barra de progreso
        progress.value = 1.0
    else:
        # Si no se ha subido una imagen
        message_label2.value = "<h2 style='color:red;'>Error: Por favor sube una imagen válida en formato .jpg o .jpeg</h2>"


In [37]:
# Asociar la función al cambio en el botón de carga
upload_button.observe(process_image, names='value')

# Mostrar los widgets 
display(udi_image_widget)  # Imagen de la UDI
display(widgets.HTML("<h1>Clasificación de imágenes</h2>"))
display(warning_label)  # Advertencia de prototipo
display(message_label)
display(upload_button)
display(output)  # Aquí se mostrará la imagen
display(progress)  # Barra de progreso
display(message_label2)  # Aquí se mostrará la predicción

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xba\x00\x00\x00s\x08\x06\x00\x00\x01\x99`\xf7\xd…

HTML(value='<h1>Clasificación de imágenes</h2>')

HTML(value="<h3 style='color:#8e1414;'>Esto es solamente un prototipo que busca un acercamiento a la clasifica…

Label(value='Por favor, sube una imagen en formato .jpg o .jpeg')

FileUpload(value=({'name': '108_right.jpeg', 'type': 'image/jpeg', 'size': 74933, 'content': <memory at 0x0000…

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 640x480 with 1 Axes>', 'i…

FloatProgress(value=1.0, description='Progreso:', layout=Layout(display='flex'), max=1.0)

HTML(value="<h2 style='color:#040cab;'>Predicción del modelo: Retinopatía Diabética</h1>")